In [5]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import matplotlib.pyplot as plt
from torch.utils.data import ConcatDataset
from PIL import Image
import os
import torchvision.models as models
import time
import copy
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import random



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [78]:
def load_data(data_dir,
                           batch_size,
                           data_type,
                           noise_type,
                           noise_percentage,                           
                           transform,                           
                           data_percentage=0.1,
                           random_seed = 1,
                           shuffle=True):
    

    noise_type = "" if noise_type == "None" else noise_type + "/"
    path = data_dir + "/" + noise_type + data_type
    print("path: ", path)
    dataset = ImageFolder(root=path, transform=transform)
    original_classes = dataset.classes 


    labels = dataset.targets
    class_to_idx = dataset.class_to_idx
    print('cti', labels, class_to_idx)

    num_samples = len(dataset)
    print("length of original dataset:", num_samples)
    indices = list(range(num_samples)) 
    if data_percentage != 1:
        needed_length = int(num_samples*data_percentage)
        while shuffle == True:
            random.shuffle(indices)
            new_indices = indices[:needed_length]
            selected_labels = [labels[idx] for idx in new_indices]
            class_names = [class_name for class_name, idx in class_to_idx.items() if idx in selected_labels]
            if len(class_names) != len(original_classes):
                print('\n Reshuffling since not all of the classes are existing in the new dataloader ==>', len(class_names), " != ", len(original_classes))
            else:
                print("it is the same")
                shuffle = False 


        indices = indices[:needed_length]
        
    
    


    length_dataset = len(indices)
    print("length of final dataset:", length_dataset)
    

    dataloader = DataLoader(dataset, batch_size=batch_size)

    # return dataloader, length_dataset, class_names
    

   

In [79]:
#Testing

data_dir = '../data/vegetable_images' 
batch_size = 64
data_type = "test"
noise_type = "None"
noise_percentage = 100                    
transform = transforms.Compose([
    transforms.Resize((227, 227)),  
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) 
])                   
data_percentage = 0.01


load_data(data_dir,
                           batch_size,
                           data_type,
                           noise_type,
                           noise_percentage,                           
                           transform,                           
                           data_percentage=data_percentage,
                           random_seed = 1,
                           shuffle=True)

# print(dataloader, l, c )

path:  ../data/vegetable_images/test
cti [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [69]:
dataloaders = {'train':  DataLoader(train_dataset, batch_size=64, shuffle=True), 
               'val': DataLoader(val_dataset, batch_size=64)
               }

dataloaders_with_noise = {'train':  DataLoader(train_dataset, batch_size=64, shuffle=True), 
               'val': DataLoader(val_dataset_with_noise, batch_size=64)
               }


test_loader = DataLoader(test_dataset, batch_size=64)
test_loader_without_noise = DataLoader(test_dataset_without_noise, batch_size=64)

NameError: name 'train_dataset' is not defined